In [1]:
!pip install pydot --quiet
!pip install gensim --quiet
!pip install tensorflow-datasets --quiet
!pip install -U tensorflow-text --quiet
!pip install transformers --quiet
!pip install jsonlines --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.10.0 requires tensorflow<2.11,>=2.10.0; platform_machine != "arm64" or platform_system != "Darwin", which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-datasets 4.9.2 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.


In [1]:
from transformers import (
    BertTokenizer,
    BertModel,
    TFBertModel,
    BertConfig,
    AutoTokenizer,
    AutoModel,
)

/home/frost/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-02 23:34:55.084822: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-02 23:34:55.109781: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-02 23:34:55.480581: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [3]:
import pandas as pd
import numpy as np
import random
import math

In [4]:
DATA_PATH = '/content/drive/MyDrive/w266_final_project/data/EmoCause'

In [5]:
# Local data path
DATA_PATH= './data/EmoCause'

In [6]:
cause_model_name = 'mrm8488/spanbert-finetuned-squadv2'

spanbert_tokenizer = BertTokenizer.from_pretrained(cause_model_name)
spanbert_model = TFBertModel.from_pretrained(cause_model_name)

2023-08-02 23:35:00.274529: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-02 23:35:00.300362: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-02 23:35:00.300471: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [7]:
vocab_size = 28996

In [8]:
bert_vocab = spanbert_tokenizer.convert_ids_to_tokens([i for i in range(vocab_size)])

In [9]:
tokenized_vocab = spanbert_tokenizer(bert_vocab,
                                     max_length=3,
                                     truncation=True,
                                     padding='max_length',
                                     return_tensors='tf')

In [10]:
token_embeds = spanbert_model(tokenized_vocab)[0][:,1,:]

In [11]:
def find_synonyms_multi(input, topn = 1):
  tokenized_input = spanbert_tokenizer(input,
                                      max_length=3,
                                      truncation=True,
                                      padding='max_length',
                                      return_tensors='tf')

  input_vec = spanbert_model(tokenized_input)[0][:,1,:]

  input_dot_embeds = np.dot(input_vec, np.transpose(token_embeds))
  input_dot_input = np.repeat(np.expand_dims(np.sum(input_vec*input_vec, axis=1),
                                            axis=1),
                              vocab_size,
                              axis=1)
  embed_dot_embed = np.sum(token_embeds*token_embeds, axis=1)

  cosine_sims = input_dot_embeds / np.sqrt(input_dot_input * embed_dot_embed)

  topn_tokens = np.argsort(-cosine_sims, axis=1)[:,1:1+topn]

  return topn_tokens, np.apply_along_axis(spanbert_tokenizer.convert_ids_to_tokens, 1, topn_tokens)

In [14]:
bert_syn_dict = {}
for i in range(14):
  syn_ids = find_synonyms_multi(bert_vocab[2000*i:2000*(i+1)])[0].squeeze()
  bert_syn_dict_batch = {2000*i + idx:e for (idx, e) in enumerate(syn_ids)}
  bert_syn_dict.update(bert_syn_dict_batch)

In [15]:
syn_ids = find_synonyms_multi(bert_vocab[28000:28996])[0].squeeze()
bert_syn_dict_batch = {28000 + idx:e for (idx, e) in enumerate(syn_ids)}
bert_syn_dict.update(bert_syn_dict_batch)

In [16]:
# Random token swap procedure:
# Grab a random row of the dataframe
# Take the tokens of the original situation
# Randomly select (without replacement) 1-5 words among the tokens
# Replace the tokens
# Add a new row to that dataframe

In [17]:
[np.argmax(text_encoding.input_ids == 0) - 1 for i in text_encoding.input_ids][0]

NameError: name 'text_encoding' is not defined

In [14]:
emocause_train_preprocessed = pd.read_csv(f'{DATA_PATH}/emocause_train_preprocessed.csv')

In [ ]:
balance_class_samples = 625 - emocause_train_preprocessed.emotion.value_counts()

In [ ]:
dict(balance_class_samples)

In [ ]:
augment = pd.DataFrame()
for k in dict(balance_class_samples):
  augment = pd.concat([augment, emocause_train_preprocessed[emocause_train_preprocessed.emotion == k].sample(balance_class_samples[k], replace=True)])

In [ ]:
augment_sit_encodings = spanbert_tokenizer(list(augment.original_situation),
                                           padding=True,
                                           return_tensors='tf')

In [ ]:
end_of_sit = [np.argmax(i == 0) - 1 for i in augment_sit_encodings.input_ids]

In [ ]:
for i in range(len(end_of_sit)):
  if end_of_sit[i] == -1:
    end_of_sit[i] = 123

In [ ]:
random_ind_select = []
for i in range(16980):
  ind_list = random.sample(range(1, end_of_sit[i]), random.sample(range(1, min(5, end_of_sit[i])), 1)[0])
  random_ind_select.append(ind_list)

In [ ]:
full_tokens_replace = []

for i in range(len(list(augment_sit_encodings['input_ids']))):
    full_tokens_replace.append(list(np.take(augment_sit_encodings['input_ids'][i], random_ind_select[i])))

In [ ]:
# Replace the tokens
new_situations = []

for i in range(len(random_ind_select)):
  a = augment_sit_encodings['input_ids'][i].numpy()
  np.put(a, random_ind_select[i], [bert_syn_dict[id] for id in full_tokens_replace[i]])

  new_situations.append(spanbert_tokenizer.convert_tokens_to_string(spanbert_tokenizer.convert_ids_to_tokens(a[1:end_of_sit[i]])))

In [ ]:
augment['original_situation'] = new_situations

In [ ]:
augment

,Unnamed: 0,original_situation,emotion,cause_span,start_index,end_index,emotion_label
1902,2130,my boyfriend proposed to me when i wasn ' t ex...,__surprised__,proposed,3,3,29
2798,1925,yet time i looking at the mirror and thought m...,__surprised__,monster,13,13,29
2731,2386,i can ' t belive your brother went on the roll...,__surprised__,brother went on the roller coaster,9,14,29
2656,1320,my husband intercepted me an beautiful diamond...,__surprised__,husband bought me a beautiful diamond 14 kt go...,2,12,29
649,2874,as i woke up today i find out it was still dar...,__surprised__,dark,12,12,29
...,...,...,...,...,...,...,...
803,273,i had been the initial factory in a ice juice ...,__faithful__,initial buyer,5,6,15
2807,2833,i has Never migrate on my girlfriend curiously...,__faithful__,never cheated,3,4,15
2839,3712,my wife for the last 10 years has been exact w...,__faithful__,loving and trusting,11,13,15
374,3151,my lover,__faithful__,wife,2,2,15


In [135]:
def insert_random_tokens(text, start_ind, end_ind):

  # Tokenize input text and grab end index.
  text_encoding = spanbert_tokenizer(text,
                                     padding=True,
                                     return_tensors='tf')

  end_of_text = text_encoding.input_ids[0].shape[0]

  # We randomly choose between 1 and num_tokens / 8 spots to insert tokens in.
  num_insertions = random.randint(1, max(1, math.ceil(16 / 8)))

  text_ids = text_encoding.input_ids[0]
  new_start_ind = start_ind
  new_end_ind = end_ind

  for i in range(num_insertions):
    # Choose a random position to insert token in.
    insertion_index = random.randint(1, end_of_text - 2)
    syn_sel_index = random.randint(1, end_of_text - 2)

    # If the token is on or before start index, add 1 to both start and end indices.
    if insertion_index <= start_ind:
      new_start_ind += 1
      new_end_ind += 1
    # If it is after start index and on/before end index, add 1 to end index only.
    elif insertion_index > start_ind and insertion_index <= end_ind:
      new_end_ind += 1
    else:
      pass

    id_ss_ind = text_encoding.input_ids.numpy()[0][syn_sel_index]

    # New token takes the place of position and moves everything to the right.
    # New token will be a synonym of random token in sentence.
    text_ids = np.insert(text_ids, insertion_index, bert_syn_dict[id_ss_ind])
    end_of_text += 1

    # Note that in this method we could add synonyms of augmented synonyms.

  new_text = spanbert_tokenizer.convert_tokens_to_string(spanbert_tokenizer.convert_ids_to_tokens(text_ids[1:end_of_text - 1]))
  return new_text, new_start_ind, new_end_ind

In [ ]:
augment_next = augment

In [ ]:
augment_next[['original_situation', 'start_index', 'end_index']] = pd.DataFrame(augment_next.apply(lambda x: insert_random_tokens(x['original_situation'], int(x['start_index']), int(x['end_index'])), axis=1, result_type='expand'))

In [ ]:
augment_next

,Unnamed: 0,original_situation,emotion,cause_span,start_index,end_index,emotion_label
1902,2130,my boyfriend proposed to ’ me when i wasn ' t ...,__surprised__,proposed,3,3,29
2798,1925,yet time i looking at m the mirror and thought...,__surprised__,monster,14,14,29
2731,2386,i can ' t belivefred your brother went on the ...,__surprised__,brother went on the roller coaster,10,15,29
2656,1320,my husband intercepted within me an [CLS] beau...,__surprised__,husband bought me a beautiful diamond 14 kt go...,2,14,29
649,2874,as i woke up he today i find out it [CLS] was ...,__surprised__,dark,14,14,29
...,...,...,...,...,...,...,...
803,273,i had been the initial factory in a ice juice ...,__faithful__,initial buyer,5,6,15
2807,2833,i never has Never migrate on my girlfriend cur...,__faithful__,never cheated,4,5,15
2839,3712,my wife for the oversized last 10 years has be...,__faithful__,loving and trusting,12,14,15
374,3151,your my lover,__faithful__,wife,3,3,15


In [ ]:
augment = pd.concat([emocause_train_preprocessed, augment])
augment = augment.sample(frac=1)
augment

,Unnamed: 0,original_situation,emotion,cause_span,start_index,end_index,emotion_label
21,2064,i even remember your old systems days ...,__nostalgic__,old college days,5,7,24
2952,3505,"yesterday i ate a whole big bag for m & ms , i...",__ashamed__,ate,3,3,6
2284,1500,when people trying in control me,__sad__,people try to control,2,5,27
653,3634,the Security Arm,__anticipating__,security guards,2,3,3
885,3484,"once in kindergarten , a pair of your underwea...",__embarrassed__,underwear,9,9,13
...,...,...,...,...,...,...,...
2963,2953,i work in a call center that has 4 members on ...,__annoyed__,queue so i am back to the top again,56,65,2
1191,480,when i was going through my old photo albums t...,__sentimental__,old photo albums,7,9,28
479,3265,today at work i addressed a male as mrs . real...,__embarrassed__,addressed a male as mrs.,5,11,13
744,2041,one time m intercepted shuttle supplies before...,__prepared__,bought emergency,4,5,25


In [ ]:
from google.colab import files
augment.to_csv('augment_3_emocause_r_bal_delins.csv')
files.download('augment_3_emocause_r_bal_delins.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
emocause_train_preprocessed

,Unnamed: 0,original_situation,emotion,cause_span,start_index,end_index,emotion_label
0,3651,one time i saw a giant cockroach under my desk!,__surprised__,giant cockroach,6,8,29
1,3375,i started a diet today and as soon as i got ho...,__guilty__,diet today and as soon as i got home i ate a b...,4,17,18
2,2609,i was late for a meeing.,__terrified__,late,3,3,30
3,308,i was buying some beer and smokes with a coupl...,__surprised__,full blown speakeasy,69,73,29
4,287,my husband was ordained to be a minister last ...,__proud__,husband was ordained to be a minister last yea...,2,27,26
...,...,...,...,...,...,...,...
3015,459,my wife and i had to move far away from home r...,__grateful__,cosign,25,26,17
3016,317,i had been working on my dance moves for the l...,__confident__,working on my dance,4,7,8
3017,1385,"when i went into labor with my youngest child,...",__terrified__,heartbeat dropped,22,23,30
3018,3026,last week there was a large spider on my wall.,__afraid__,large spider on my wall,6,10,0


### Random Token Deletion

In [50]:
first_ex = emocause_train_preprocessed.iloc[0]
first_ex

Unnamed: 0                                                       3651
original_situation    one time i saw a giant cockroach under my desk!
emotion                                                 __surprised__
cause_span                                            giant cockroach
start_index                                                         6
end_index                                                           8
emotion_label                                                      29
Name: 0, dtype: object

In [51]:
first_ex['original_situation']

'one time i saw a giant cockroach under my desk!'

In [166]:
'''
text: The raw string to process

start_ind: The start index of the target span (indexing starts from 1 not 0)

end_ind: The end index of the target span (indexing starts from 1 not 0)

tokenizer: The tokenizer object to use to tokenize the input

p_delete: Probability that we delete a token

inside_target_range: If set, will allow for random deletion of tokens inside the target span (otherwise avoids them)
'''

def delete_random_tokens(text, start_ind,
                         end_ind, tokenizer=None,
                         p_delete=0.2, inside_target_range=False):

    if tokenizer is None:
        tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    # Convert index to start at 0 (instead of 1)
    start_ind -= 1
    end_ind -= 1

    # Determine number of tokens to delete
    tokens = tokenizer.tokenize(text)
    num_tokens = len(tokens)

    # Compute the range of indices to allow deletion
    allowed_indices = list(range(num_tokens))
    if inside_target_range:
        allowed_indices = [i for i in allowed_indices if not (start_ind <= i <= end_ind)]

    # Ensure we delete at least one token
    num_tokens_to_delete = max(1, int(len(allowed_indices) * p_delete))

    # Randomly determine tokens for deletion
    deleted_indices = random.sample(allowed_indices, num_tokens_to_delete)

    # Process deleted indices to adjust start & end appropriately
    for i in sorted(deleted_indices, reverse=True):
        if i < start_ind:
            start_ind -= 1
            end_ind -= 1
        elif i == start_ind:
            start_ind += 1
        elif i == end_ind:
            end_ind -= 1

    # Convert index back to starting at 1 (instead of 0)
    start_ind += 1
    end_ind += 1

    # Copy over only tokens that were not "deleted"
    new_tokens = [token for i, token in enumerate(tokens) if i not in deleted_indices]
    new_text = tokenizer.convert_tokens_to_string(new_tokens)

    return new_text, start_ind, end_ind

#### Example: Run the random token deletion util on the same span 3 times

In [142]:
text_with_deletion = []
new_indices = []
for _ in range(3):
    new_text, new_start, new_end = delete_random_tokens(first_ex['original_situation'], 
                                                        first_ex['start_index'],
                                                        first_ex['end_index'],
                                                        spanbert_tokenizer)
    text_with_deletion.append(new_text)
    new_indices.append((new_start, new_end))

print(text_with_deletion)
print(new_indices)

['one time saw a giant cockroach under my desk', 'one saw a giant cockroach under my desk !', 'one i saw a giant cock under my desk !']
[(5, 7), (4, 6), (5, 6)]


C:\Users\Devin\AppData\Local\Temp\ipykernel_36024\2111491049.py:41: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  deleted_indices = set(random.sample(allowed_indices, num_tokens_to_delete))


#### Example: Now allow target span to be included in deletion (we might not want to allow this for context)

In [91]:
text_with_deletion = []
new_indices = []
for _ in range(3):
    new_text, new_start, new_end = delete_random_tokens(first_ex['original_situation'], 
                                                        first_ex['start_index'],
                                                        first_ex['end_index'],
                                                        spanbert_tokenizer,
                                                        inside_target_range=True)
    text_with_deletion.append(new_text)
    new_indices.append((new_start, new_end))

print(text_with_deletion)
print(new_indices)

['one time i saw a cockroach under my desk', 'one time i saw a giantroach under my desk', 'one time i saw a giant cock under my !']
[(7, 8), (6, 8), (6, 7)]


#### Apply to dataset

In [143]:
augment_deletion = emocause_train_preprocessed.apply(
    lambda row: delete_random_tokens(
        row['original_situation'],
        row['start_index'],
        row['end_index'],
        spanbert_tokenizer
    ),
    axis=1
)

C:\Users\Devin\AppData\Local\Temp\ipykernel_36024\2111491049.py:41: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  deleted_indices = set(random.sample(allowed_indices, num_tokens_to_delete))


In [144]:
augment_deletion

0        (one time i saw cockroach under my desk !, 6, 7)
1       (started a and as soon as i got home i ate a b...
2                         (i was late for meeing ., 3, 3)
3       (i was buying some and smokes with couple of f...
4       (husband was ordained to be minister last year...
                              ...                        
3015    (my wife and i had to move away from for her g...
3016    (i had been working on my dance moves for the ...
3017    (when i went labor with my youngest child , wh...
3018      (last week there was a large spider on ., 6, 8)
3019    (i had a math test and i knew would ace it ., ...
Length: 3020, dtype: object

#### Parse back into dataframe

In [145]:
df_augument_deletion = emocause_train_preprocessed.copy()

In [148]:
# Populate the mutated columns
mutated_tuples = augment_deletion.values.tolist()

for i, row in df_augument_deletion.iterrows():
    curr_tuple = mutated_tuples[i]
    text, start, end = curr_tuple
    
    df_augument_deletion.at[i, 'original_situation'] = text
    df_augument_deletion.at[i, 'start_index'] = str(start)
    df_augument_deletion.at[i, 'end_index'] = str(end)

In [149]:
df_augument_deletion

,Unnamed: 0,original_situation,emotion,cause_span,start_index,end_index,emotion_label
0,3651,one time i saw cockroach under my desk !,__surprised__,giant cockroach,6,7,29
1,3375,started a and as soon as i got home i ate a bu...,__guilty__,diet today and as soon as i got home i ate a b...,4,16,18
2,2609,i was late for meeing .,__terrified__,late,3,3,30
3,308,i was buying some and smokes with couple of fr...,__surprised__,full blown speakeasy,57,60,29
4,287,husband was ordained to be minister last year ...,__proud__,husband was ordained to be a minister last yea...,1,26,26
...,...,...,...,...,...,...,...
3015,459,my wife and i had to move away from for her gr...,__grateful__,cosign,21,22,17
3016,317,i had been working on my dance moves for the l...,__confident__,working on my dance,4,7,8
3017,1385,"when i went labor with my youngest child , whi...",__terrified__,heartbeat dropped,20,21,30
3018,3026,last week there was a large spider on .,__afraid__,large spider on my wall,6,8,0


#### Compare the deleted strings and original strings

In [150]:
rand_indices = random.sample([i for i in range(len(emocause_train_preprocessed))], 5)
for i in rand_indices:
    before_str = emocause_train_preprocessed['original_situation'].iloc[i]
    after_str = df_augument_deletion['original_situation'].iloc[i]
    print(f'BEFORE:\n\t{before_str}')
    print(f'AFTER:\n\t{after_str}')
    print('==========')

BEFORE:
	leaving the bar the other day with my boyfriend, i totally tripped and fell on the step just outside the door and scraped my knee. so humiliating.
AFTER:
	the bar the day with boyfriend , i totally tripped fell on the step just outside the door and scraped knee . so humting .
BEFORE:
	my dog keeps having accidents in the house ever since we got another dog. i keep taking her outside to!
AFTER:
	my dog having accidents in house ever we got another dog . i taking her outside to !
BEFORE:
	i was walking back to my desk after getting a cup of coffee and tripped over my own two feet and spilled coffee all over myself 
AFTER:
	walking back to my desk getting a cup of coffee and tripped over my own two feet and coffee all myself
BEFORE:
	when i went through one of my unpacked boxes, i found some of my ex-girlfriend's old clothes that she never picked up. i really missed her, and i started thinking about all the good times we enjoyed together.
AFTER:
	when went through one my unpacked

### Balance the dataset by using random insertion and deletion to augment the data

In [21]:
emocause_train_preprocessed = pd.read_csv(f'{DATA_PATH}/emocause_train_preprocessed.csv')

In [22]:
balance_class_samples = 625 - emocause_train_preprocessed.emotion.value_counts()

In [23]:
augment_r_insert_delete = pd.DataFrame()
for k in dict(balance_class_samples):
  augment_r_insert_delete = pd.concat([augment_r_insert_delete, emocause_train_preprocessed[emocause_train_preprocessed.emotion == k].sample(balance_class_samples[k], replace=True)])

#### Alternate between random insertion and deletion

In [85]:
import pandas as pd

# Make a copy of the original DataFrame before applying any mutations
augment_r_insert_delete_copy = augment_r_insert_delete.copy()

emotions = set(emocause_train_preprocessed.emotion.tolist())

for emotion in emotions:
    # Get all rows with this class
    emotion_rows = augment_r_insert_delete[augment_r_insert_delete['emotion'] == emotion]

    # Drop duplicate indices to ensure unique indices
    emotion_rows_unique_index = emotion_rows.loc[~emotion_rows.index.duplicated(keep='first')]

    # Randomly partition rows for insertion or deletion
    rand_insert_rows = emotion_rows_unique_index.sample(frac=0.5).copy()
    rand_delete_rows = emotion_rows_unique_index.drop(rand_insert_rows.index).copy()

    # Apply random insertion
    rand_insert_rows[['original_situation', 'start_index', 'end_index']] = rand_insert_rows.apply(
        lambda x: pd.Series(insert_random_tokens(x['original_situation'], 
                                                  int(x['start_index']), 
                                                  int(x['end_index']))), 
        axis=1
    )

    # Apply random deletion to the other half
    rand_delete_rows[['original_situation', 'start_index', 'end_index']] = rand_delete_rows.apply(
        lambda x: pd.Series(delete_random_tokens(x['original_situation'], 
                                                  int(x['start_index']), 
                                                  int(x['end_index']),
                                                  spanbert_tokenizer)), 
        axis=1
    )

    # Create a new DataFrame with NaN values for each row
    mutated_rows_template = pd.DataFrame(index=emotion_rows_unique_index.index, columns=augment_r_insert_delete.columns)

    # Update the mutated_rows_template with the actual mutated rows
    mutated_rows_template.update(rand_insert_rows)
    mutated_rows_template.update(rand_delete_rows)

    # Update the original DataFrame with the mutated rows
    augment_r_insert_delete.update(mutated_rows_template)


#### Merge the augmented data back in

In [103]:
emocause_augmented_rand_insert_delete = pd.concat([augment_r_insert_delete, emocause_train_preprocessed], axis=0)
emocause_augmented_rand_insert_delete.drop('Unnamed: 0', axis=1, inplace=True)
emocause_augmented_rand_insert_delete

,original_situation,emotion,cause_span,start_index,end_index,emotion_label
948,i so happy to come home the other day and ever...,__surprised__,deserve,24.0,24.0,29
1484,my surprised me on my birthday with an amazing...,__surprised__,homemade dinner,10.0,12.0,29
345,i was by easy it was to quit smoking !,__surprised__,quit smoking,8.0,9.0,29
939,"for my anniversary at got my job , all my cowo...",__surprised__,coworkers threw me a party,11.0,17.0,29
2254,my mom told me at christmas that we would be g...,__surprised__,cruise vacation,16.0,17.0,29
...,...,...,...,...,...,...
3015,my wife and i had to move far away from home r...,__grateful__,cosign,25,26,17
3016,i had been working on my dance moves for the l...,__confident__,working on my dance,4,7,8
3017,"when i went into labor with my youngest child,...",__terrified__,heartbeat dropped,22,23,30
3018,last week there was a large spider on my wall.,__afraid__,large spider on my wall,6,10,0


#### Verify class balancing

In [66]:
from collections import defaultdict

def check_class_balance(df):
    emo_count = defaultdict(int)
    
    for emotion in df.emotion:
        emo_count[emotion] += 1
    
    return emo_count

In [93]:
check_class_balance(augment_r_insert_delete)

defaultdict(int,
            {'__surprised__': 473,
             '__annoyed__': 511,
             '__impressed__': 514,
             '__sentimental__': 516,
             '__sad__': 518,
             '__excited__': 518,
             '__embarrassed__': 521,
             '__disgusted__': 521,
             '__proud__': 523,
             '__jealous__': 523,
             '__afraid__': 525,
             '__nostalgic__': 526,
             '__disappointed__': 529,
             '__terrified__': 531,
             '__prepared__': 532,
             '__angry__': 533,
             '__anxious__': 533,
             '__joyful__': 534,
             '__lonely__': 534,
             '__grateful__': 535,
             '__confident__': 536,
             '__ashamed__': 537,
             '__furious__': 537,
             '__content__': 537,
             '__caring__': 540,
             '__hopeful__': 541,
             '__apprehensive__': 542,
             '__anticipating__': 544,
             '__trusting__': 546,


In [94]:
check_class_balance(emocause_train_preprocessed)

defaultdict(int,
            {'__surprised__': 152,
             '__guilty__': 69,
             '__terrified__': 94,
             '__proud__': 102,
             '__grateful__': 90,
             '__afraid__': 100,
             '__furious__': 88,
             '__disgusted__': 104,
             '__joyful__': 91,
             '__prepared__': 93,
             '__ashamed__': 88,
             '__anxious__': 92,
             '__trusting__': 79,
             '__embarrassed__': 104,
             '__annoyed__': 114,
             '__excited__': 107,
             '__nostalgic__': 99,
             '__disappointed__': 96,
             '__confident__': 89,
             '__faithful__': 59,
             '__sad__': 107,
             '__caring__': 85,
             '__devastated__': 77,
             '__apprehensive__': 83,
             '__sentimental__': 109,
             '__jealous__': 102,
             '__content__': 88,
             '__lonely__': 91,
             '__impressed__': 111,
             '__an

In [104]:
check_class_balance(emocause_augmented_rand_insert_delete)

defaultdict(int,
            {'__surprised__': 625,
             '__annoyed__': 625,
             '__impressed__': 625,
             '__sentimental__': 625,
             '__sad__': 625,
             '__excited__': 625,
             '__embarrassed__': 625,
             '__disgusted__': 625,
             '__proud__': 625,
             '__jealous__': 625,
             '__afraid__': 625,
             '__nostalgic__': 625,
             '__disappointed__': 625,
             '__terrified__': 625,
             '__prepared__': 625,
             '__angry__': 625,
             '__anxious__': 625,
             '__joyful__': 625,
             '__lonely__': 625,
             '__grateful__': 625,
             '__confident__': 625,
             '__ashamed__': 625,
             '__furious__': 625,
             '__content__': 625,
             '__caring__': 625,
             '__hopeful__': 625,
             '__apprehensive__': 625,
             '__anticipating__': 625,
             '__trusting__': 625,


In [105]:
emocause_augmented_rand_insert_delete.to_csv('./data/AugmentExperiments/augment_3_emocause_r_balanced_insert_delete.csv')

#### Merge EmoCause and GNE training data, append augmentations

In [194]:
gne_train_preprocessed = pd.read_csv('./data/GoodNewsEveryone/gne_train_preprocessed.csv')
gne_train_preprocessed

,Unnamed: 0,headline,dominant_emotion,cause,cause_span,start_index,end_index,emotion_label
0,3466,delta airlines refuses to boycott georgia over...,disgust,"[[""to boycott georgia over abortion laws""]]",to boycott georgia over abortion laws,4,11,2
1,1848,french special forces rescue hostages in a dea...,joy,"[[""rescue hostages in a deadly burkina faso ra...",rescue hostages in a deadly burkina faso raid,5,17,5
2,3195,"palestinian government sworn in, again, after ...",negative_surprise,"[[""sworn in, again, after botched oath""]]","sworn in, again, after botched oath",5,14,8
3,3683,cleric: muslims ‘have a duty’ to obtain nuclea...,negative_anticipation_including_pessimism,"[[""to obtain nuclear weapons""]]",to obtain nuclear weapons,14,17,7
4,194,usa today sports attacks on 'raging' trump app...,disgust,"[[""appear to be coordinated media campaign""]]",appear to be coordinated media campaign,12,17,2
...,...,...,...,...,...,...,...,...
3067,4777,the big drop in corporate tax rates and revenues,negative_surprise,"[[""tax rates and revenues""]]",tax rates and revenues,6,9,8
3068,2018,letters: maybe dan fagan should read hemingway...,positive_anticipation_including_optimism,"[[""maybe dan fagan should read hemingway and t...",maybe dan fagan should read hemingway and thoreau,3,16,9
3069,2557,"pope in morocco: walls, fear-mongering won't s...",fear,"[[""walls, fear-mongering won't stop migration""]]","walls, fear-mongering won't stop migration",7,17,3
3070,3161,house republicans ask democrats to help force ...,positive_surprise,"[[""help force vote on protecting lives of babi...",help force vote on protecting lives of babies ...,8,18,10


In [196]:
emocause_train_preprocessed = pd.read_csv('./data/EmoCause/emocause_train_preprocessed.csv')

In [197]:
emocause_train_preprocessed.drop(columns=['Unnamed: 0'], inplace=True)
emocause_train_preprocessed

,original_situation,emotion,cause_span,start_index,end_index,emotion_label
0,one time i saw a giant cockroach under my desk!,__surprised__,giant cockroach,6,8,29
1,i started a diet today and as soon as i got ho...,__guilty__,diet today and as soon as i got home i ate a b...,4,17,18
2,i was late for a meeing.,__terrified__,late,3,3,30
3,i was buying some beer and smokes with a coupl...,__surprised__,full blown speakeasy,69,73,29
4,my husband was ordained to be a minister last ...,__proud__,husband was ordained to be a minister last yea...,2,27,26
...,...,...,...,...,...,...
3015,my wife and i had to move far away from home r...,__grateful__,cosign,25,26,17
3016,i had been working on my dance moves for the l...,__confident__,working on my dance,4,7,8
3017,"when i went into labor with my youngest child,...",__terrified__,heartbeat dropped,22,23,30
3018,last week there was a large spider on my wall.,__afraid__,large spider on my wall,6,10,0


#### Rename columns, map GNE emotions to EmoCause's emotions

In [198]:
gne_train_remapped =  gne_train_preprocessed.rename(columns={
    'headline': 'original_situation',
    'dominant_emotion': 'emotion'
})
gne_train_remapped.drop(columns=['cause', 'Unnamed: 0'], inplace=True) 
gne_train_remapped

,original_situation,emotion,cause_span,start_index,end_index,emotion_label
0,delta airlines refuses to boycott georgia over...,disgust,to boycott georgia over abortion laws,4,11,2
1,french special forces rescue hostages in a dea...,joy,rescue hostages in a deadly burkina faso raid,5,17,5
2,"palestinian government sworn in, again, after ...",negative_surprise,"sworn in, again, after botched oath",5,14,8
3,cleric: muslims ‘have a duty’ to obtain nuclea...,negative_anticipation_including_pessimism,to obtain nuclear weapons,14,17,7
4,usa today sports attacks on 'raging' trump app...,disgust,appear to be coordinated media campaign,12,17,2
...,...,...,...,...,...,...
3067,the big drop in corporate tax rates and revenues,negative_surprise,tax rates and revenues,6,9,8
3068,letters: maybe dan fagan should read hemingway...,positive_anticipation_including_optimism,maybe dan fagan should read hemingway and thoreau,3,16,9
3069,"pope in morocco: walls, fear-mongering won't s...",fear,"walls, fear-mongering won't stop migration",7,17,3
3070,house republicans ask democrats to help force ...,positive_surprise,help force vote on protecting lives of babies ...,8,18,10


In [199]:
# Remap emotions
gne_train_remapped.emotion = gne_train_remapped.emotion.map({
    'anger': '__angry__',
    'annoyance': '__annoyed__',
    'disgust': '__disgusted__',
    'fear': '__afraid__',
    'guilt': '__guilty__',
    'joy': '__joyful__',
    'love_including_like': '__content__',
    'negative_anticipation_including_pessimism': '__apprehensive__',
    'negative_surprise': '__disappointed__',
    'positive_anticipation_including_optimism': '__excited__',
    'positive_surprise': '__impressed__',
    'pride': '__proud__',
    'sadness': '__sad__',
    'shame': '__ashamed__',
    'trust': '__trusting__'
})
gne_train_remapped

,original_situation,emotion,cause_span,start_index,end_index,emotion_label
0,delta airlines refuses to boycott georgia over...,__disgusted__,to boycott georgia over abortion laws,4,11,2
1,french special forces rescue hostages in a dea...,__joyful__,rescue hostages in a deadly burkina faso raid,5,17,5
2,"palestinian government sworn in, again, after ...",__disappointed__,"sworn in, again, after botched oath",5,14,8
3,cleric: muslims ‘have a duty’ to obtain nuclea...,__apprehensive__,to obtain nuclear weapons,14,17,7
4,usa today sports attacks on 'raging' trump app...,__disgusted__,appear to be coordinated media campaign,12,17,2
...,...,...,...,...,...,...
3067,the big drop in corporate tax rates and revenues,__disappointed__,tax rates and revenues,6,9,8
3068,letters: maybe dan fagan should read hemingway...,__excited__,maybe dan fagan should read hemingway and thoreau,3,16,9
3069,"pope in morocco: walls, fear-mongering won't s...",__afraid__,"walls, fear-mongering won't stop migration",7,17,3
3070,house republicans ask democrats to help force ...,__impressed__,help force vote on protecting lives of babies ...,8,18,10


In [200]:
# Verify emotion set 
emocause_emotions = set(emocause_train_preprocessed['emotion'].tolist())
gne_emotions = set(gne_train_remapped['emotion'].tolist())

assert len(emocause_emotions.union(gne_emotions)) == len(emocause_emotions)

In [201]:
# Convenience map of EmoCause label to emotion label
label_to_emotion_text = {29: '__surprised__',
 18: '__guilty__',
 30: '__terrified__',
 26: '__proud__',
 17: '__grateful__',
 0: '__afraid__',
 16: '__furious__',
 12: '__disgusted__',
 22: '__joyful__',
 25: '__prepared__',
 6: '__ashamed__',
 4: '__anxious__',
 31: '__trusting__',
 13: '__embarrassed__',
 2: '__annoyed__',
 14: '__excited__',
 24: '__nostalgic__',
 11: '__disappointed__',
 8: '__confident__',
 15: '__faithful__',
 27: '__sad__',
 7: '__caring__',
 10: '__devastated__',
 5: '__apprehensive__',
 28: '__sentimental__',
 21: '__jealous__',
 9: '__content__',
 23: '__lonely__',
 20: '__impressed__',
 1: '__angry__',
 3: '__anticipating__',
 19: '__hopeful__'}

In [202]:
set(gne_train_remapped['emotion_label'].tolist())

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}

In [203]:
# Remap emotion labels

gne_train_remapped.emotion_label = gne_train_remapped.emotion.map({
    '__angry__': 1,
    '__annoyed__': 2,
    '__disgusted__': 12,
    '__afraid__': 0,
    '__guilty__': 18,
    '__joyful__': 22,
    '__content__': 9,
    '__apprehensive__': 5,
    '__disappointed__': 11,
    '__excited__': 14,
    '__impressed__': 20,
    '__proud__': 26,
    '__sad__': 27,
    '__ashamed__': 6,
    '__trusting__': 31
})

set(gne_train_remapped['emotion_label'].tolist())

{0, 1, 2, 5, 6, 9, 11, 12, 14, 18, 20, 22, 26, 27, 31}

In [204]:
emocause_gne_train = pd.concat([emocause_train_preprocessed, gne_train_remapped])
emocause_gne_train

,original_situation,emotion,cause_span,start_index,end_index,emotion_label
0,one time i saw a giant cockroach under my desk!,__surprised__,giant cockroach,6,8,29
1,i started a diet today and as soon as i got ho...,__guilty__,diet today and as soon as i got home i ate a b...,4,17,18
2,i was late for a meeing.,__terrified__,late,3,3,30
3,i was buying some beer and smokes with a coupl...,__surprised__,full blown speakeasy,69,73,29
4,my husband was ordained to be a minister last ...,__proud__,husband was ordained to be a minister last yea...,2,27,26
...,...,...,...,...,...,...
3067,the big drop in corporate tax rates and revenues,__disappointed__,tax rates and revenues,6,9,11
3068,letters: maybe dan fagan should read hemingway...,__excited__,maybe dan fagan should read hemingway and thoreau,3,16,14
3069,"pope in morocco: walls, fear-mongering won't s...",__afraid__,"walls, fear-mongering won't stop migration",7,17,0
3070,house republicans ask democrats to help force ...,__impressed__,help force vote on protecting lives of babies ...,8,18,20


In [205]:
emocause_gne_train.to_csv('./data/AugmentExperiments/extra/emocause_append_gne_unbalanced.csv')

### Apply all augmentations and balance the dataset

In [206]:
emocause_gne_train.emotion.value_counts()

emotion
__disappointed__    676
__impressed__       471
__angry__           377
__afraid__          359
__sad__             349
__disgusted__       332
__annoyed__         313
__excited__         307
__apprehensive__    303
__joyful__          257
__guilty__          181
__ashamed__         162
__trusting__        153
__surprised__       152
__proud__           144
__content__         119
__sentimental__     109
__embarrassed__     104
__jealous__         102
__nostalgic__        99
__terrified__        94
__prepared__         93
__anxious__          92
__lonely__           91
__grateful__         90
__confident__        89
__furious__          88
__caring__           85
__hopeful__          84
__anticipating__     81
__devastated__       77
__faithful__         59
Name: count, dtype: int64

In [207]:
balance_class_samples = math.ceil(20000 / len(np.unique(emocause_gne_train.emotion))) - emocause_gne_train.emotion.value_counts()
balance_class_samples

emotion
__disappointed__    -51
__impressed__       154
__angry__           248
__afraid__          266
__sad__             276
__disgusted__       293
__annoyed__         312
__excited__         318
__apprehensive__    322
__joyful__          368
__guilty__          444
__ashamed__         463
__trusting__        472
__surprised__       473
__proud__           481
__content__         506
__sentimental__     516
__embarrassed__     521
__jealous__         523
__nostalgic__       526
__terrified__       531
__prepared__        532
__anxious__         533
__lonely__          534
__grateful__        535
__confident__       536
__furious__         537
__caring__          540
__hopeful__         541
__anticipating__    544
__devastated__      548
__faithful__        566
Name: count, dtype: int64

In [208]:
# Perform minimal augmentation on the dominant class
balance_class_samples['__disappointed__'] = 51

#### Apply random swap augmentation

In [209]:
balance_class_samples

emotion
__disappointed__     51
__impressed__       154
__angry__           248
__afraid__          266
__sad__             276
__disgusted__       293
__annoyed__         312
__excited__         318
__apprehensive__    322
__joyful__          368
__guilty__          444
__ashamed__         463
__trusting__        472
__surprised__       473
__proud__           481
__content__         506
__sentimental__     516
__embarrassed__     521
__jealous__         523
__nostalgic__       526
__terrified__       531
__prepared__        532
__anxious__         533
__lonely__          534
__grateful__        535
__confident__       536
__furious__         537
__caring__          540
__hopeful__         541
__anticipating__    544
__devastated__      548
__faithful__        566
Name: count, dtype: int64

In [210]:
import math
# Allocate a third of the augmentation to random swap
balance_class_samples_rand_swap = {}

# The remaining values for augmentation after random swap
balance_class_samples_after_rand_swap = {}

for emotion, augment_count in balance_class_samples.items():
    allocated_examples = math.floor(augment_count / 3)
    balance_class_samples_rand_swap[emotion] = allocated_examples

    # Subtract from our required augmentation counts
    balance_class_samples_after_rand_swap[emotion] = augment_count - allocated_examples

# Update the new augmentation counts
balance_class_samples = balance_class_samples_after_rand_swap
balance_class_samples_rand_swap

{'__disappointed__': 17,
 '__impressed__': 51,
 '__angry__': 82,
 '__afraid__': 88,
 '__sad__': 92,
 '__disgusted__': 97,
 '__annoyed__': 104,
 '__excited__': 106,
 '__apprehensive__': 107,
 '__joyful__': 122,
 '__guilty__': 148,
 '__ashamed__': 154,
 '__trusting__': 157,
 '__surprised__': 157,
 '__proud__': 160,
 '__content__': 168,
 '__sentimental__': 172,
 '__embarrassed__': 173,
 '__jealous__': 174,
 '__nostalgic__': 175,
 '__terrified__': 177,
 '__prepared__': 177,
 '__anxious__': 177,
 '__lonely__': 178,
 '__grateful__': 178,
 '__confident__': 178,
 '__furious__': 179,
 '__caring__': 180,
 '__hopeful__': 180,
 '__anticipating__': 181,
 '__devastated__': 182,
 '__faithful__': 188}

In [211]:
balance_class_samples_after_rand_swap

{'__disappointed__': 34,
 '__impressed__': 103,
 '__angry__': 166,
 '__afraid__': 178,
 '__sad__': 184,
 '__disgusted__': 196,
 '__annoyed__': 208,
 '__excited__': 212,
 '__apprehensive__': 215,
 '__joyful__': 246,
 '__guilty__': 296,
 '__ashamed__': 309,
 '__trusting__': 315,
 '__surprised__': 316,
 '__proud__': 321,
 '__content__': 338,
 '__sentimental__': 344,
 '__embarrassed__': 348,
 '__jealous__': 349,
 '__nostalgic__': 351,
 '__terrified__': 354,
 '__prepared__': 355,
 '__anxious__': 356,
 '__lonely__': 356,
 '__grateful__': 357,
 '__confident__': 358,
 '__furious__': 358,
 '__caring__': 360,
 '__hopeful__': 361,
 '__anticipating__': 363,
 '__devastated__': 366,
 '__faithful__': 378}

In [221]:
augment = pd.DataFrame()
for k in dict(balance_class_samples_rand_swap):
  augment = pd.concat([augment, emocause_gne_train[emocause_gne_train.emotion == k].sample(balance_class_samples_rand_swap[k], replace=True)])

In [222]:
MAX_SEQUENCE_LENGTH = 124
augment_sit_encodings = spanbert_tokenizer(list(augment['original_situation']),
                                           padding='max_length',
                                           max_length=MAX_SEQUENCE_LENGTH,
                                           return_tensors='tf')

In [223]:
end_of_sit = [np.argmax(i == 0) - 1 for i in augment_sit_encodings.input_ids]

In [224]:
for i in range(len(end_of_sit)):
  if end_of_sit[i] == -1:
    end_of_sit[i] = 123

In [225]:
random_ind_select = []
for i in range(len(augment)):
  ind_list = random.sample(range(1, end_of_sit[i]), random.sample(range(1, min(5, end_of_sit[i])), 1)[0])
  random_ind_select.append(ind_list)

In [226]:
full_tokens_replace = []

for i in range(len(list(augment_sit_encodings['input_ids']))):
    full_tokens_replace.append(list(np.take(augment_sit_encodings['input_ids'][i], random_ind_select[i])))

In [227]:
# Replace the tokens
new_headlines = []

for i in range(len(random_ind_select)):
  a = augment_sit_encodings['input_ids'][i].numpy()
  np.put(a, random_ind_select[i], [bert_syn_dict[id] for id in full_tokens_replace[i]])

  new_headlines.append(spanbert_tokenizer.convert_tokens_to_string(spanbert_tokenizer.convert_ids_to_tokens(a[1:end_of_sit[i]])))

In [228]:
augment['original_situation'] = new_headlines

In [229]:
augment_swap = augment
augment_swap

,original_situation,emotion,cause_span,start_index,end_index,emotion_label
91,"that guantánamo case , u . s . government says...",__disappointed__,detain anyone — even u.s. citizens,17,26,11
2919,i was so hungry for lunch today i ordered pane...,__disappointed__,took over a hour,17,20,11
391,greta gerwig and noah fauactionsch tag their i...,__disappointed__,welcome their first child,13,16,11
2799,"amactions ' s stock extends bucket , tumbles 9...",__disappointed__,tumbles 9.8% after q2 results,9,19,11
2870,alan dershowitz pens panicky Nation - ed in an...,__disappointed__,panicky op-ed in anticipation of a new yorker ...,9,31,11
...,...,...,...,...,...,...
303,i has beactionsactions that my family,__faithful__,belives in my family,3,8,15
2657,i always buy the same brand of virtuesgurt and...,__faithful__,life,40,40,15
1506,i have tried my best in always do what was bes...,__faithful__,tried my best to always do what is best for my...,3,14,15
439,my ex boyfriend and i was together for 3 years...,__faithful__,boyfriend and i were together for 4 years. i w...,3,14,15


In [230]:
augment_swap.to_csv('./data/AugmentExperiments/extra/emocause_append_gne_rand_swap.csv')

#### Perform random insertion augmentation

In [ ]:
import math
# Allocate a third of the augmentation to random swap
balance_class_samples_rand_swap = {}

# The remaining values for augmentation after random swap
balance_class_samples_after_rand_swap = {}

for emotion, augment_count in balance_class_samples.items():
    allocated_examples = math.floor(augment_count / 3)
    balance_class_samples_rand_swap[emotion] = allocated_examples

    # Subtract from our required augmentation counts
    balance_class_samples_after_rand_swap[emotion] = augment_count - allocated_examples

# Update the new augmentation counts
balance_class_samples = balance_class_samples_after_rand_swap
balance_class_samples_rand_swap

In [132]:
# Allocate anotherthird of the augmentation to random swap
balance_class_samples_rand_insert = balance_class_samples_rand_swap

balance_class_after_rand_insert = {}

for emotion, prev_total in balance_class_samples.items():
    allocated_examples = balance_class_samples_rand_insert[emotion]
    balance_class_after_rand_insert[emotion] = prev_total - allocated_examples

balance_class_after_rand_insert

{'__disappointed__': 17,
 '__impressed__': 52,
 '__angry__': 84,
 '__afraid__': 90,
 '__sad__': 92,
 '__disgusted__': 99,
 '__annoyed__': 104,
 '__excited__': 106,
 '__apprehensive__': 108,
 '__joyful__': 124,
 '__guilty__': 148,
 '__ashamed__': 155,
 '__trusting__': 158,
 '__surprised__': 159,
 '__proud__': 161,
 '__content__': 170,
 '__sentimental__': 172,
 '__embarrassed__': 175,
 '__jealous__': 175,
 '__nostalgic__': 176,
 '__terrified__': 177,
 '__prepared__': 178,
 '__anxious__': 179,
 '__lonely__': 178,
 '__grateful__': 179,
 '__confident__': 180,
 '__furious__': 179,
 '__caring__': 180,
 '__hopeful__': 181,
 '__anticipating__': 182,
 '__devastated__': 184,
 '__faithful__': 190}

In [134]:
balance_class_samples_rand_insert

{'__disappointed__': 17,
 '__impressed__': 51,
 '__angry__': 82,
 '__afraid__': 88,
 '__sad__': 92,
 '__disgusted__': 97,
 '__annoyed__': 104,
 '__excited__': 106,
 '__apprehensive__': 107,
 '__joyful__': 122,
 '__guilty__': 148,
 '__ashamed__': 154,
 '__trusting__': 157,
 '__surprised__': 157,
 '__proud__': 160,
 '__content__': 168,
 '__sentimental__': 172,
 '__embarrassed__': 173,
 '__jealous__': 174,
 '__nostalgic__': 175,
 '__terrified__': 177,
 '__prepared__': 177,
 '__anxious__': 177,
 '__lonely__': 178,
 '__grateful__': 178,
 '__confident__': 178,
 '__furious__': 179,
 '__caring__': 180,
 '__hopeful__': 180,
 '__anticipating__': 181,
 '__devastated__': 182,
 '__faithful__': 188}

In [ ]:
balance_class_after_rand_insert

In [146]:
total = 0
for val in list(balance_class_samples_rand_insert.values()):
    total += val
total

4659

In [150]:
augment_insert = pd.DataFrame()
for k in dict(balance_class_samples_rand_insert):
  augment_insert = pd.concat([augment_insert, emocause_gne_train[emocause_gne_train.emotion == k].sample(balance_class_samples_rand_insert[k], replace=True)])

In [151]:
augment_insert

,original_situation,emotion,cause_span,start_index,end_index,emotion_label
89,"man charged with 64 felony counts after 1,000 ...",__disappointed__,"charged with 64 felony counts after 1,000 guns...",2,20,11
1227,looks like donald trump’s longtime accounting ...,__disappointed__,accounting firm is selling him,10,14,11
2840,the mothers fighting to get their children bac...,__disappointed__,children back from lebanon,7,12,11
2534,chart of the day: q1 growth is looking pretty ...,__disappointed__,is looking pretty anemic,9,14,11
1627,bryce harper bats down frosty dc homecoming wi...,__disappointed__,bats down frosty dc homecoming with home run,6,16,11
...,...,...,...,...,...,...
2000,everyday i wake up and see my girlfriends beau...,__faithful__,girlfriends beautiful,8,10,15
284,i'm glad i stuck my job out as long as i did -...,__faithful__,offered a promotion,19,21,15
778,"i'm loyal to my principles, that's why i do no...",__faithful__,"loyal to my principles, that's why i do not ea...",4,17,15
1305,one time i thought about getting dominoes pizz...,__faithful__,loyal,18,18,15


In [152]:
augment_insert[['original_situation', 'start_index', 'end_index']] = pd.DataFrame(augment_insert.apply(lambda x: insert_random_tokens(x['original_situation'], int(x['start_index']), int(x['end_index'])), axis=1, result_type='expand'))

In [153]:
augment_insert

,original_situation,emotion,cause_span,start_index,end_index,emotion_label
89,man charged with 64 felony counts after 1 desp...,__disappointed__,"charged with 64 felony counts after 1,000 guns...",2,22,11
1227,looks like don walkald trump ’ s longtime acco...,__disappointed__,accounting firm is selling him,11,15,11
2840,the his mothers fighting to get battling their...,__disappointed__,children back from lebanon,9,14,11
2534,chart of the day lock : q1 growth is looking p...,__disappointed__,is looking pretty anemic,10,15,11
1627,bryactionsce harper bats down frosty dc homeco...,__disappointed__,bats down frosty dc homecoming with home run,7,17,11
...,...,...,...,...,...,...
2000,everyday i wake up and see them my girlfriends...,__faithful__,girlfriends beautiful,9,11,15
284,i ' m glad i stuck my job out as long gotten a...,__faithful__,offered a promotion,20,22,15
778,"i ' m ’ loyal to my principles , that ' s why ...",__faithful__,"loyal to my principles, that's why i do not ea...",5,19,15
1305,one time i been thought about getting due domi...,__faithful__,loyal,20,20,15


In [154]:
augment_insert.to_csv('./data/AugmentExperiments/extra/emocause_append_gne_rand_insert.csv')

In [156]:
emocause_gne_rand_swap_insert_train = pd.concat([emocause_gne_rand_swap_train, augment_insert])
emocause_gne_rand_swap_insert_train

,original_situation,emotion,cause_span,start_index,end_index,emotion_label
807,rep . matt gactionstz allegedly hit by former ...,__disappointed__,former rival in ‘milkshaking’ incident,12,20,11
1324,‘ na kidding ’ ! john dean helpfulactions poin...,__disappointed__,‘facts’ are not a priority at house judiciary,15,24,11
696,david beckactions gets Six - month driving ban...,__disappointed__,gets six-month driving ban for using phone at ...,6,16,11
2912,deputy reassigned with assaulting his 73 - sea...,__disappointed__,charged with assaulting his 73-year-old mother...,2,21,11
457,wti oil futures mark lowest settlement since k...,__disappointed__,wti oil futures mark lowest settlement,1,8,11
...,...,...,...,...,...,...
2000,everyday i wake up and see them my girlfriends...,__faithful__,girlfriends beautiful,9,11,15
284,i ' m glad i stuck my job out as long gotten a...,__faithful__,offered a promotion,20,22,15
778,"i ' m ’ loyal to my principles , that ' s why ...",__faithful__,"loyal to my principles, that's why i do not ea...",5,19,15
1305,one time i been thought about getting due domi...,__faithful__,loyal,20,20,15


In [157]:
emocause_gne_rand_swap_insert_train.to_csv('./data/AugmentExperiments/extra/emocause_append_gne_rand_swap_insert.csv')

#### Perform random deletion augmentation

In [159]:
# Use remaining allocations
balance_class_samples_rand_deletion = balance_class_after_rand_insert
balance_class_samples_rand_deletion

{'__disappointed__': 17,
 '__impressed__': 52,
 '__angry__': 84,
 '__afraid__': 90,
 '__sad__': 92,
 '__disgusted__': 99,
 '__annoyed__': 104,
 '__excited__': 106,
 '__apprehensive__': 108,
 '__joyful__': 124,
 '__guilty__': 148,
 '__ashamed__': 155,
 '__trusting__': 158,
 '__surprised__': 159,
 '__proud__': 161,
 '__content__': 170,
 '__sentimental__': 172,
 '__embarrassed__': 175,
 '__jealous__': 175,
 '__nostalgic__': 176,
 '__terrified__': 177,
 '__prepared__': 178,
 '__anxious__': 179,
 '__lonely__': 178,
 '__grateful__': 179,
 '__confident__': 180,
 '__furious__': 179,
 '__caring__': 180,
 '__hopeful__': 181,
 '__anticipating__': 182,
 '__devastated__': 184,
 '__faithful__': 190}

In [162]:
augment_delete = pd.DataFrame()
for k in dict(balance_class_samples_rand_deletion):
  augment_delete = pd.concat([augment_delete, emocause_gne_train[emocause_gne_train.emotion == k].sample(balance_class_samples_rand_deletion[k], replace=True)])

augment_delete

,original_situation,emotion,cause_span,start_index,end_index,emotion_label
1711,what disasters reveal about mental-health care,__disappointed__,disasters reveal about mental-health care,2,8,11
672,the middle east is now split between red state...,__disappointed__,split between red states and blue states,6,12,11
3040,phrma is funding a democratic think tank tryin...,__disappointed__,think tank trying to derail medicare for all,8,18,11
1276,vaping: the astonishing surge in e-cigarette use,__disappointed__,the astonishing surge in e-cigarette use,5,14,11
2745,"london bridge was like a war zone, says fiance...",__disappointed__,london bridge was like a war zone,1,8,11
...,...,...,...,...,...,...
401,a co-worker has been flirting with me at work....,__faithful__,co-worker has been flirting with me at work. i...,2,18,15
2475,i needed help after i had surgery with househo...,__faithful__,always so true,28,30,15
1919,a girl i used to date asked me to come and vis...,__faithful__,asked me to come and visit her for a one night...,7,25,15
2377,"one time i wanted to eat at burger king, but t...",__faithful__,"wanted to eat at burger king, but then i remem...",4,19,15


In [168]:
augment_delete[['original_situation', 'start_index', 'end_index']] = pd.DataFrame(augment_delete.apply(lambda x: delete_random_tokens(
    x['original_situation'], 
    int(x['start_index']), 
    int(x['end_index']), 
    spanbert_tokenizer), 
                                                                                                       axis=1, result_type='expand'))

In [169]:
augment_delete

,original_situation,emotion,cause_span,start_index,end_index,emotion_label
1711,what disasters reveal about - health care,__disappointed__,disasters reveal about mental-health care,2,8,11
672,the middle east is now split between red state...,__disappointed__,split between red states and blue states,6,11,11
3040,phrma is a think tank trying to derail medicar...,__disappointed__,think tank trying to derail medicare for all,6,16,11
1276,vaping : astonishing surge e - cigarette use,__disappointed__,the astonishing surge in e-cigarette use,6,14,11
2745,"london bridge was a war zone , says fiance ter...",__disappointed__,london bridge was like a war zone,1,8,11
...,...,...,...,...,...,...
401,a co - worker has flirting with me at . i ' ve...,__faithful__,co-worker has been flirting with me at work. i...,2,17,15
2475,help after surgery with household chores and m...,__faithful__,always so true,22,24,15
1919,a girl i used to asked me to come visit her fo...,__faithful__,asked me to come and visit her for a one night...,6,24,15
2377,one time wanted to eat at burger king but then...,__faithful__,"wanted to eat at burger king, but then i remem...",3,18,15


In [ ]:
emocause_gne_rand_swap_insert_train.to_csv('./data/AugmentExperiments/extra/emocause_append_gne_rand_swap_insert.csv')

In [170]:
augment_delete.to_csv('./data/AugmentExperiments/extra/emocause_append_gne_rand_delete.csv')

In [172]:
augment_insert

,original_situation,emotion,cause_span,start_index,end_index,emotion_label
374,my wife,__faithful__,wife,2,2,15
1919,a girl i used to date asked me to come and vis...,__faithful__,asked me to come and visit her for a one night...,7,25,15
1331,when i am in nature i feel good that this worl...,__faithful__,nature,5,5,15
723,i'm glad i can trust my husband to always be t...,__faithful__,trust my husband,7,9,15
682,"once i was going to get fried chicken, but the...",__faithful__,loyal to only eating vegetables,16,20,15
...,...,...,...,...,...,...
303,i have belives in my family,__faithful__,belives in my family,3,8,15
2553,i have to rely on my prayers and my belief in ...,__faithful__,money,25,25,15
1342,last week i was at the grocery store picking u...,__faithful__,brand,32,32,15
262,my boyfriend is going out with a bunch of peop...,__faithful__,boyfriend is going out with a bunch of people ...,2,28,15


In [174]:
emocause_append_gne_rand_swap_insert_delete = pd.concat([emocause_gne_rand_swap_train, augment_insert, augment_delete])
emocause_append_gne_rand_swap_insert_delete

,original_situation,emotion,cause_span,start_index,end_index,emotion_label,Unnamed: 0
807,rep . matt gactionstz allegedly hit by former ...,__disappointed__,former rival in ‘milkshaking’ incident,12,20,11,NaN
1324,‘ na kidding ’ ! john dean helpfulactions poin...,__disappointed__,‘facts’ are not a priority at house judiciary,15,24,11,NaN
696,david beckactions gets Six - month driving ban...,__disappointed__,gets six-month driving ban for using phone at ...,6,16,11,NaN
2912,deputy reassigned with assaulting his 73 - sea...,__disappointed__,charged with assaulting his 73-year-old mother...,2,21,11,NaN
457,wti oil futures mark lowest settlement since k...,__disappointed__,wti oil futures mark lowest settlement,1,8,11,NaN
...,...,...,...,...,...,...,...
401,a co - worker has flirting with me at . i ' ve...,__faithful__,co-worker has been flirting with me at work. i...,2,17,15,NaN
2475,help after surgery with household chores and m...,__faithful__,always so true,22,24,15,NaN
1919,a girl i used to asked me to come visit her fo...,__faithful__,asked me to come and visit her for a one night...,6,24,15,NaN
2377,one time wanted to eat at burger king but then...,__faithful__,"wanted to eat at burger king, but then i remem...",3,18,15,NaN


#### Concat all augmented data together

In [235]:
augment_insert = pd.read_csv('./data/AugmentExperiments/extra/emocause_append_gne_rand_insert.csv')
augment_delete = pd.read_csv('./data/AugmentExperiments/extra/emocause_append_gne_rand_delete.csv')
augment_swap = pd.read_csv('./data/AugmentExperiments/extra/emocause_append_gne_rand_swap.csv')
emocause_gne = pd.read_csv('./data/AugmentExperiments/extra/emocause_append_gne_unbalanced.csv')

In [236]:
emocause_gne

,Unnamed: 0,original_situation,emotion,cause_span,start_index,end_index,emotion_label
0,0,one time i saw a giant cockroach under my desk!,__surprised__,giant cockroach,6,8,29
1,1,i started a diet today and as soon as i got ho...,__guilty__,diet today and as soon as i got home i ate a b...,4,17,18
2,2,i was late for a meeing.,__terrified__,late,3,3,30
3,3,i was buying some beer and smokes with a coupl...,__surprised__,full blown speakeasy,69,73,29
4,4,my husband was ordained to be a minister last ...,__proud__,husband was ordained to be a minister last yea...,2,27,26
...,...,...,...,...,...,...,...
6087,3067,the big drop in corporate tax rates and revenues,__disappointed__,tax rates and revenues,6,9,11
6088,3068,letters: maybe dan fagan should read hemingway...,__excited__,maybe dan fagan should read hemingway and thoreau,3,16,14
6089,3069,"pope in morocco: walls, fear-mongering won't s...",__afraid__,"walls, fear-mongering won't stop migration",7,17,0
6090,3070,house republicans ask democrats to help force ...,__impressed__,help force vote on protecting lives of babies ...,8,18,20


In [237]:
emocause_gne_augmented_all_train = pd.concat([
    augment_insert,
    augment_delete,
    augment_swap,
    emocause_gne
])
emocause_gne_augmented_all_train

,Unnamed: 0,original_situation,emotion,cause_span,start_index,end_index,emotion_label
0,89,man charged with 64 felony counts after 1 desp...,__disappointed__,"charged with 64 felony counts after 1,000 guns...",2,22,11
1,1227,looks like don walkald trump ’ s longtime acco...,__disappointed__,accounting firm is selling him,11,15,11
2,2840,the his mothers fighting to get battling their...,__disappointed__,children back from lebanon,9,14,11
3,2534,chart of the day lock : q1 growth is looking p...,__disappointed__,is looking pretty anemic,10,15,11
4,1627,bryactionsce harper bats down frosty dc homeco...,__disappointed__,bats down frosty dc homecoming with home run,7,17,11
...,...,...,...,...,...,...,...
6087,3067,the big drop in corporate tax rates and revenues,__disappointed__,tax rates and revenues,6,9,11
6088,3068,letters: maybe dan fagan should read hemingway...,__excited__,maybe dan fagan should read hemingway and thoreau,3,16,14
6089,3069,"pope in morocco: walls, fear-mongering won't s...",__afraid__,"walls, fear-mongering won't stop migration",7,17,0
6090,3070,house republicans ask democrats to help force ...,__impressed__,help force vote on protecting lives of babies ...,8,18,20


In [238]:
emocause_gne_augmented_all_train.to_csv('./data/AugmentExperiments/augment_4_emocause_w_gne_balanced.csv')